In [1]:
import unidecode
import distance
import pandas as pd
import numpy as np
import re
import math
from collections import Counter

In [2]:
df = pd.read_csv('target_schema.csv')

In [3]:
# Xóa dấu tiếng việt và ký tự thừa
def preprocess(text):
    # text = text.lower()
    text = unidecode.unidecode(text)
    text = re.sub('[^a-zA-Z0-9 \n\.]', ' ', text)
    text = text.replace('nan', '')
    
    return text

In [4]:
# Lấy bản ghi giống nhất
def min_d(text,copus):
    d = sorted(distance.ilevenshtein(text, copus))
    nearest = []
    indices = [i for i, v in enumerate(d) if v[0] == min(d)[0]]
    for i in indices:
        nearest.append(d[i])
        
    return nearest


In [5]:
# Tìm category của input
def compare_cate(text):
    category = ['iphone', 'samsung', 'xiaomi', 'oppo', 'vsmart', 'realme', 'vivo','nokia', 'huawei']
    
    cate = ''
    for cate in category: 
        if cate in text: # đúng cú pháp
            cate = cate
        else: # sai cú pháp
            cate = min_d(text, category)[0][1]

    cols = ['name', 'ram', 'memory/rom']
    name = df['name'][df['category'] == cate]
    full_name = df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)[df['category'] == cate]

    return name, full_name


In [6]:
# Sentence to vector
def text_to_vec(text):
    word = re.compile(r'\w+')
    word = word.findall(text)
    
    return Counter(word)

In [7]:
t1 = 'điện thoại iphone 12 pro max 128gb'
t1 = preprocess(t1)
v1 = text_to_vec(t1)
v1

Counter({'dien': 1,
         'thoai': 1,
         'iphone': 1,
         '12': 1,
         'pro': 1,
         'max': 1,
         '128gb': 1})

In [8]:
# Độ đo Cosine
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [9]:
# Hàm tìm kiếm
def compare_item(text):
    more_similar_item = []

    text = preprocess(text)
    vec1 = text_to_vec(text)
    name, full_name = compare_cate(text)

    for i in range(len(name)):
        vec2 = text_to_vec(preprocess((list(full_name)[i])))
        cosine = get_cosine(vec1, vec2)
        if cosine > 0.7:
            more_similar_item.append([cosine, list(name)[i]])

    if more_similar_item == []:
        print('Hãy nhập thêm thông tin để tìm kiếm chính xác hơn!')
        more_similar_item = name

    return sorted(more_similar_item, reverse= True)

In [10]:
# Caching
def catching(input_, INPUT, OUTPUT):
    min = 100000
    for i in INPUT:
        d = distance.levenshtein(preprocess(input_), preprocess(i)) # So sánh input mới với các input cũ
        if d < min:
            min = d
            I = i
    
    if min < 10:
        print('------ Use Catching Function ------')
        OUTPUT.append(OUTPUT[INPUT.index(I)])
        INPUT.append(input_)
        index = INPUT.index(I)

    else:
        OUTPUT.append(compare_item(input_))
        INPUT.append(input_)
        index = INPUT.index(input_)

    return INPUT, OUTPUT, index

In [11]:
# Tìm kiếm
def search(input_, INPUT, OUTPUT):

    if len(INPUT) == 0:
        INPUT = [input_]
        OUTPUT = []
        OUTPUT.append(compare_item(input_))
        index = 0
    
    elif len(INPUT) == 10:
        INPUT = INPUT[1:]
        OUTPUT = OUTPUT[1:]
        INPUT, OUTPUT, index = catching(input_, INPUT, OUTPUT)

    else:
        INPUT, OUTPUT, index = catching(input_, INPUT, OUTPUT)
        
    return INPUT, OUTPUT, index

In [12]:
INPUT = []
OUTPUT = []

In [13]:
input_ = input()
INPUT, OUTPUT, index = search(input_, INPUT, OUTPUT)

# print(INPUT)
OUTPUT

# for i in range(len(INPUT)):
#     if type(OUTPUT[i][0]) is str:
#         print('Hệ thống gợi ý ----->', i+1, OUTPUT[i][0], '<-----', INPUT[i])
#     else:
#         print(i+1, OUTPUT[i][0][1], '<-----', INPUT[i])

[[[0.7745966692414834, 'vsmart active 3 6gb ram']]]

In [14]:
# TEST
'''
1   Mùa hè sôi động :v
2   11 pro max 64gb iphone chính hãng
3   iphon 12 pro max 126 gb chính hang vn/q
4   samsung note 10 5g
5   samsug galaxy s21 128gb utra
6   xiaomi redmi note 10 128 gb
7   xeomi redmi not 8 pro 8gb/128g
8   Tìm cái j đi anh
9   asus rog phone 3 chính hãng
10  vsmart joy 4 4gb
11  apple iphone x 64gb quốc tế cũ 99%(ll/a)
12  pple iphone 11 pro max 128 gb chính gãng
'''

'\n1   Mùa hè sôi động :v\n2   11 pro max 64gb iphone chính hãng\n3   iphon 12 pro max 126 gb chính hang vn/q\n4   samsung note 10 5g\n5   samsug galaxy s21 128gb utra\n6   xiaomi redmi note 10 128 gb\n7   xeomi redmi not 8 pro 8gb/128g\n8   Tìm cái j đi anh\n9   asus rog phone 3 chính hãng\n10  vsmart joy 4 4gb\n11  apple iphone x 64gb quốc tế cũ 99%(ll/a)\n12  pple iphone 11 pro max 128 gb chính gãng\n'